In [3]:
import os
os.environ["THEANO_FLAGS"] = 'cuda.root=/usr/local/cuda,floatX=float32,device=gpu1,force_device=False,lib.cnmem=.75'

import theano
print(theano.config.device)

import mhcflurry, seaborn, numpy, pandas, pickle, sklearn, collections, scipy, time
import mhcflurry.dataset
import fancyimpute, locale

import sklearn.metrics
import sklearn.cross_validation

def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')

gpu1


In [4]:
max_ic50 = 50000
min_peptides_to_consider_allele = 10
data_dir = "../data/"

In [6]:
all_train_data = mhcflurry.dataset.Dataset.from_csv(data_dir + "bdata.2009.mhci.public.1.txt")

In [7]:
imputed_train_data = all_train_data.impute_missing_values(
    fancyimpute.MICE(n_imputations=25, n_burn_in=5),
    min_observations_per_peptide=2,
    min_observations_per_allele=10
)

Dropping 12235 peptides with <2 observations
Dropping 9 alleles with <10 observations: ['ELA-A1', 'HLA-B2701', 'HLA-B3508', 'HLA-B44', 'HLA-E0101', 'Mamu-B04', 'Patr-A0602', 'Patr-B0901', 'Patr-B1701']
[MICE] Completing matrix with shape (19304, 97)
[MICE] Starting imputation round 1/30, elapsed time 0.048
[MICE] Starting imputation round 2/30, elapsed time 4.002
[MICE] Starting imputation round 3/30, elapsed time 7.712
[MICE] Starting imputation round 4/30, elapsed time 11.453
[MICE] Starting imputation round 5/30, elapsed time 15.168
[MICE] Starting imputation round 6/30, elapsed time 18.849
[MICE] Starting imputation round 7/30, elapsed time 22.609
[MICE] Starting imputation round 8/30, elapsed time 26.438
[MICE] Starting imputation round 9/30, elapsed time 30.400
[MICE] Starting imputation round 10/30, elapsed time 34.912
[MICE] Starting imputation round 11/30, elapsed time 39.564
[MICE] Starting imputation round 12/30, elapsed time 47.226
[MICE] Starting imputation round 13/30, el

In [8]:
validation_df = pandas.read_csv("../data/combined_test_BLIND_dataset_from_kim2013.csv")
validation_df

,allele,peptide,length,meas,netmhc,netmhcpan,smmpmbec_cpp
0,H-2-DB,AAACNVATA,9,657.657837,154.881662,711.213514,438.530698
1,H-2-DB,AAFEFVYV,8,30831.879502,6456.542290,785.235635,10351.421667
2,H-2-DB,AAFVNDYSL,9,77.446180,17.458222,7.516229,28.054336
3,H-2-DB,AAIANQAAV,9,1.999862,9.638290,9.749896,25.703958
4,H-2-DB,AAIANQAVV,9,1.517050,8.550667,8.336812,28.773984
5,H-2-DB,AAIENYVRF,9,37.844258,252.348077,114.815362,187.068214
6,H-2-DB,AAINFITTM,9,3.155005,199.986187,389.045145,200.909281
7,H-2-DB,AAIPAPPPI,9,3243.396173,1059.253725,493.173804,295.120923
8,H-2-DB,AAKLNRPPL,9,654.636174,66.374307,77.268059,38.459178
9,H-2-DB,AALDMVDAL,9,229.614865,547.015963,597.035287,225.423921


In [14]:
validation_allele_counts = validation_df.allele.value_counts()
train_allele_counts = all_train_data._df.allele.value_counts()
validation_allele_counts
train_allele_counts

HLA-A0201     9565
HLA-A0301     6141
HLA-A0203     5542
HLA-A1101     5399
HLA-A0206     4827
HLA-A3101     4796
HLA-A6802     4768
HLA-A0202     3919
HLA-A0101     3725
HLA-B0702     3412
H-2-KB        3407
H-2-DB        3216
HLA-B1501     3213
HLA-A6801     3184
HLA-A3301     3040
HLA-B2705     3028
HLA-A2601     2894
HLA-B4001     2718
HLA-B5801     2564
HLA-A2402     2533
HLA-B3501     2397
HLA-A2902     2397
HLA-B0801     2267
Mamu-A01      2264
HLA-A6901     2079
HLA-B1801     2052
HLA-A3001     2040
HLA-A2301     2021
HLA-B5701     1857
HLA-B5101     1734
              ... 
HLA-B3801      136
HLA-A0250      132
HLA-B7301      115
HLA-A11         74
HLA-A0207       68
HLA-B7          64
HLA-A0205       56
Mamu-A07        53
Mamu-A2601      51
HLA-A2          44
HLA-B5802       42
HLA-A0210       18
Gogo-B0101      15
ELA-A1          14
HLA-A0302       10
Patr-B1701       8
HLA-B3503        7
HLA-A6601        4
HLA-B2702        4
HLA-A26          4
HLA-B1402        3
HLA-B4201   

In [23]:
alleles = sorted(train_allele_counts.index[
    (train_allele_counts >= min_peptides_to_consider_allele)
    & (train_allele_counts.index.isin(validation_allele_counts.index))
], key=lambda allele: -1 * train_allele_counts[allele])
alleles

['HLA-A0201',
 'HLA-A0301',
 'HLA-A0203',
 'HLA-A1101',
 'HLA-A0206',
 'HLA-A3101',
 'HLA-A6802',
 'HLA-A0202',
 'HLA-A0101',
 'HLA-B0702',
 'H-2-KB',
 'H-2-DB',
 'HLA-B1501',
 'HLA-A6801',
 'HLA-A3301',
 'HLA-B2705',
 'HLA-A2601',
 'HLA-B4001',
 'HLA-B5801',
 'HLA-A2402',
 'HLA-B3501',
 'HLA-A2902',
 'HLA-B0801',
 'Mamu-A01',
 'HLA-A6901',
 'HLA-B1801',
 'HLA-A3001',
 'HLA-A2301',
 'HLA-B5701',
 'HLA-B5101',
 'HLA-B4402',
 'HLA-A3002',
 'HLA-B4601',
 'HLA-B5401',
 'HLA-B5301',
 'Mamu-A02',
 'HLA-B4403',
 'HLA-B4501',
 'HLA-B3901',
 'HLA-B4002',
 'HLA-B1517',
 'HLA-A8001',
 'HLA-A3201',
 'HLA-A2501',
 'HLA-B0802',
 'H-2-KD',
 'HLA-B2703',
 'HLA-B1503',
 'HLA-B1509',
 'HLA-B0803',
 'HLA-A2603',
 'HLA-A2602',
 'HLA-B3801']

In [24]:
dropout_probabilities = [0.0, 0.5]
embedding_output_dims_and_layer_sizes_list = [(32, [64]), (8, [4])]
activations = ["tanh"]

models_params_list = []

for model_num in range(1):
    for impute in [False, True]:
        for dropout_probability in dropout_probabilities:
            for (embedding_output_dim, layer_sizes) in embedding_output_dims_and_layer_sizes_list:
                for activation in activations:
                    models_params_list.append(dict(
                        impute=impute,
                        dropout_probability=dropout_probability,  
                        embedding_output_dim=embedding_output_dim,
                        layer_sizes=layer_sizes,
                        activation=activation))

print("%d models" % len(models_params_list))
models_params_explored = set.union(*[set(x) for x in models_params_list])
models_params_explored


8 models


{'activation',
 'dropout_probability',
 'embedding_output_dim',
 'impute',
 'layer_sizes'}

In [32]:
def log_to_ic50(log_value):
        """
        Convert neural network output to IC50 values between 0.0 and
        self.max_ic50 (typically 5000, 20000 or 50000)
        """
        return max_ic50 ** (1.0 - log_value)

def make_scores(ic50_y, ic50_y_pred, sample_weight=None, threshold_nm=500):     
    y_pred = mhcflurry.regression_target.ic50_to_regression_target(ic50_y_pred, max_ic50)
    try:
        auc = sklearn.metrics.roc_auc_score(ic50_y <= threshold_nm, y_pred, sample_weight=sample_weight)
    except ValueError:
        auc = numpy.nan
    try:
        f1 = sklearn.metrics.f1_score(ic50_y <= threshold_nm, ic50_y_pred <= threshold_nm, sample_weight=sample_weight)
    except ValueError:
        f1 = numpy.nan
    try:
        tau = scipy.stats.kendalltau(ic50_y_pred, ic50_y)[0]
    except ValueError:
        tau = numpy.nan
    
    return dict(
        auc=auc,
        f1=f1,
        tau=tau,
    )    

def mean_with_std(grouped_column, decimals=3):
    pattern = "%%0.%df" % decimals
    return pandas.Series([
        (pattern + " +/ " + pattern) % (m, s) if not pandas.isnull(s) else pattern % m
        for (m, s) in zip(grouped_column.mean(), grouped_column.std())
    ], index = grouped_column.mean().index)

def allele_data_to_df(data):
    d = data._asdict()
    d["X_index"] = [x for x in d["X_index"]]
    d["X_binary"] = [x for x in d["X_binary"]]
    df = pandas.DataFrame(d).set_index('peptides')
    return df

def make_2d_array(thing):
    return numpy.array([list(x) for x in thing])

def df_to_allele_data(df):
    d = dict((col, df[col].values) for col in df)
    d["X_index"] = make_2d_array(d["X_index"])
    (d["max_ic50"],) = list(df.max_ic50.unique())
    return mhcflurry.data.AlleleData(peptides = df.index.values, **d)


In [33]:
models_and_scores = {}
validation_df_with_mhcflurry = validation_df.copy()


In [35]:
# train and test models, adding columns to validation_df_with_mhcflurry
pandas.DataFrame(models_params_list).to_csv("../data/validation_models.csv", index=False)

def make_and_fit_model(allele, original_params):
    params = dict(original_params)
    impute = params["impute"]
    del params["impute"]
    model = mhcflurry.Class1BindingPredictor.from_hyperparameters(max_ic50=max_ic50, **params)
    print("Fitting model for allele %s (%d + %d): %s" % (
            allele,
            len(all_train_data.groupby_allele_dictionary()[allele]),
            len(imputed_train_data.groupby_allele_dictionary()[allele]),
            str(original_params)))
    t = -time.time()
    model.fit_dataset(
        all_train_data.groupby_allele_dictionary()[allele],
        pretraining_dataset=imputed_train_data.groupby_allele_dictionary()[allele] if impute else None,
        verbose=False,
        batch_size=128,
        n_training_epochs=250)
    t += time.time()
    print("Trained in %d sec" % t)
    return model

for (i, allele) in enumerate(alleles):
    if allele not in validation_df_with_mhcflurry.allele.unique():
        print("Skipping allele %s: not in test set" % allele)
        continue
    if allele in models_and_scores:
        print("Skipping allele %s: already done" % allele)
        continue
    values_for_allele = []
    for (j, params) in enumerate(models_params_list):
        print("Allele %d model %d" % (i, j))
        model = make_and_fit_model(allele, params)
        predictions = model.predict(
            list(validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele == allele].peptide))
        print("test set size: %d" % len(predictions))
        validation_df_with_mhcflurry.loc[(validation_df_with_mhcflurry.allele == allele),
                                         ("mhcflurry %d" % j)] = predictions
        scores = make_scores(validation_df_with_mhcflurry.ix[validation_df.allele == allele].meas,
                            predictions)
        print(scores)
        values_for_allele.append((params, scores))
        
    models_and_scores[allele] = values_for_allele
    
    # Write out all data after each allele.
    validation_df_with_mhcflurry_results = validation_df_with_mhcflurry.ix[validation_df_with_mhcflurry.allele.isin(models_and_scores)]
    validation_df_with_mhcflurry_results.to_csv("../data/validation_predictions_full.csv", index=False)
    
    scores_df = collections.defaultdict(list)
    predictors = validation_df_with_mhcflurry_results.columns[4:]

    for (allele, grouped) in validation_df_with_mhcflurry_results.groupby("allele"):
        scores_df["allele"].append(allele)
        scores_df["test_size"].append(len(grouped.meas))
        for predictor in predictors:
            scores = make_scores(grouped.meas, grouped[predictor])
            for (key, value) in scores.items():
                scores_df["%s_%s" % (predictor, key)].append(value)

    scores_df = pandas.DataFrame(scores_df)
    scores_df["train_size"] = [
        len(all_train_data.groupby_allele_dictionary()[a])
        for a in scores_df.allele
    ]

    scores_df.index = scores_df.allele
    scores_df.to_csv("../data/validation_scores.csv", index=False)
        


Skipping allele HLA-A0201: already done
Allele 1 model 0
Fitting model for allele HLA-A0301 (6141 + 19304): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.0, 'layer_sizes': [64]}
Trained in 50 sec
test set size: 811
{'tau': 0.43438214718912999, 'auc': 0.82528648306426089, 'f1': 0.78222222222222226}
Allele 1 model 1
Fitting model for allele HLA-A0301 (6141 + 19304): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 8, 'dropout_probability': 0.0, 'layer_sizes': [4]}
Trained in 22 sec
test set size: 811
{'tau': 0.58213187535133193, 'auc': 0.92022475466919906, 'f1': 0.86752136752136744}
Allele 1 model 2
Fitting model for allele HLA-A0301 (6141 + 19304): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.5, 'layer_sizes': [64]}
Trained in 99 sec
test set size: 811
{'tau': 0.60037775507791202, 'auc': 0.92539411206077871, 'f1': 0.87272727272727268}
Allele 1 model 3
Fitting model for allele HL

/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)



{'tau': 0.36312329401344945, 'auc': 0.98585652862638606, 'f1': 0.0}
Allele 45 model 0
Fitting model for allele H-2-KD (452 + 19304): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.0, 'layer_sizes': [64]}
Trained in 7 sec
test set size: 229
{'tau': 0.3421177599435396, 'auc': 0.76339804214368678, 'f1': 0.55714285714285716}
Allele 45 model 1
Fitting model for allele H-2-KD (452 + 19304): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 8, 'dropout_probability': 0.0, 'layer_sizes': [4]}
Trained in 4 sec
test set size: 229
{'tau': 0.3525982229128653, 'auc': 0.77227476356396207, 'f1': 0.562962962962963}
Allele 45 model 2
Fitting model for allele H-2-KD (452 + 19304): {'activation': 'tanh', 'impute': False, 'embedding_output_dim': 32, 'dropout_probability': 0.5, 'layer_sizes': [64]}
Trained in 11 sec
test set size: 229
{'tau': 0.36831891736685379, 'auc': 0.78621204579392745, 'f1': 0.59259259259259256}
Allele 45 model 3
Fitting mode

In [36]:
print_full(scores_df[["train_size", "test_size"]].sort("train_size", inplace=False))

           train_size  test_size
allele                          
HLA-B3801         136        351
HLA-A2602         202        413
HLA-A2603         205        312
HLA-B0803         217        234
HLA-B1509         346        466
HLA-B1503         429        165
HLA-B2703         433        441
H-2-KD            452        229
HLA-B0802         487        509
HLA-A2501         519        416
HLA-A3201         640        449
HLA-A8001         782        379
HLA-B1517         846        582
HLA-B4002         866         74
HLA-B3901         886        641
HLA-B4501         889         65
HLA-B4403         913        378
Mamu-A02          938        388
HLA-B5301        1018        485
HLA-B5401        1019         79
HLA-B4601        1424        378
HLA-A3002        1430        420
HLA-B4402        1705        411
HLA-B5101        1734        854
HLA-B5701        1857        815
HLA-A2301        2021        391
HLA-A3001        2040        660
HLA-B1801        2052        503
HLA-A6901 